In [1]:
import numpy as np
import os
from utils.utilities import *
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import tensorflow as tf

In [3]:
print(tf.__version__)

1.3.0


## Load data

In [4]:
DATASET_PATH = "./dataset/UCI HAR Dataset/"

In [5]:
INPUT_SIGNAL_TYPES = [
    "body_acc_x_",
    "body_acc_y_",
    "body_acc_z_",
    "body_gyro_x_",
    "body_gyro_y_",
    "body_gyro_z_",
    "total_acc_x_",
    "total_acc_y_",
    "total_acc_z_"
]

In [6]:
def load_X(X_signals_paths):
    X_signals = []
    
    for signal_type_path in X_signals_paths:
        with open(signal_type_path, "r") as f:
            X_signals.append(
                [np.array(serie, dtype=np.float32)
                    for serie in [row.replace('  ', ' ').strip().split(' ') for row in f]]
            )
    return np.transpose(X_signals, (1, 2, 0))

def load_y(y_path):

    # Read dataset from disk, dealing with text file's syntax
    with open(y_path, "r") as f:
        y = np.array(
            [elem for elem in [
                row.replace('  ', ' ').strip().split(' ') for row in f
            ]], 
            dtype=np.int32
        )
            
    y = y.reshape(-1,)
    # Substract 1 to each output class for friendly 0-based indexing 
    return y - 1

In [7]:
X_train_signals_paths = [
    DATASET_PATH + "train/Inertial Signals/" + signal + "train.txt" for signal in INPUT_SIGNAL_TYPES
]
X_test_signals_paths = [
    DATASET_PATH + "test/Inertial Signals/" + signal + "test.txt" for signal in INPUT_SIGNAL_TYPES
]

In [8]:
har_X_train = load_X(X_train_signals_paths)
har_X_test = load_X(X_test_signals_paths)

In [9]:
y_train_path = DATASET_PATH + "train/y_train.txt"
y_test_path = DATASET_PATH + "test/y_test.txt"

In [10]:
har_y_train = load_y(y_train_path)
har_y_test = load_y(y_test_path)

## Prepare data

In [11]:
X_train, X_valid, y_train, y_valid = train_test_split(har_X_train, har_y_train, random_state=42)

In [12]:
X_train.shape, y_train.shape

((5514, 128, 9), (5514,))

In [13]:
X_train = np.transpose(X_train, (0, 2, 1))
X_valid = np.transpose(X_valid, (0, 2, 1))
har_X_test = np.transpose(har_X_test, (0, 2, 1))

In [14]:
X_train.shape, y_train.shape

((5514, 9, 128), (5514,))

# Build LSTM model

## Hyperparameters

In [15]:
batch_size = 200
n_epochs = 100
learning_rate = 0.0005

seq_len = 128
n_channels = 9
n_neurons = 100
n_layers = 3

# outputs
n_classes = 6

In [16]:
n_batches = X_train.shape[0] // batch_size

## a simple LSTM model

In [17]:
tf.reset_default_graph()
tf.set_random_seed(73)
np.random.seed(73)

X = tf.placeholder(tf.float32, (None, n_channels, seq_len), name="X")
y = tf.placeholder(tf.int32, (None), name="y")

# is in training phase
training = tf.placeholder_with_default(False, shape=(), name="training")


In [18]:
with tf.name_scope("rnn"):
    
    basic_cell = tf.contrib.rnn.BasicLSTMCell(num_units=n_neurons,
                                             activation=tf.nn.relu)
    outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)
    

    top_layer_h_state = states[-1]
    #print(top_layer_h_state.shape)
    states_concat = tf.concat(axis=1, values=states)
    logits = tf.layers.dense(states_concat, n_classes)

In [19]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy, name="loss")

In [20]:
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)

In [21]:
with tf.name_scope("eval"):
    #correct = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [22]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [23]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for i in range(n_batches):
            X_batch = X_train[i * batch_size:(i+1) * batch_size]
            y_batch = y_train[i * batch_size:(i+1) * batch_size]
            train_acc =  sess.run([training_op], feed_dict={X: X_batch, y: y_batch, training: True})
        
        if epoch % 10 == 0:
            valid_acc, loss_val = sess.run([accuracy, loss], feed_dict={X: X_valid, y: y_valid})
            print("Epoch: ", epoch,"Loss:", loss_val, "Valid accuracy: ", valid_acc)
    saver.save(sess, "./model/har_first_lstm_model.ckpt")

Epoch:  0 Loss: 1.10706 Valid accuracy:  0.624048
Epoch:  10 Loss: 0.221656 Valid accuracy:  0.90914
Epoch:  20 Loss: 0.189022 Valid accuracy:  0.934168
Epoch:  30 Loss: 0.186363 Valid accuracy:  0.93852
Epoch:  40 Loss: 0.206757 Valid accuracy:  0.942329
Epoch:  50 Loss: 0.237743 Valid accuracy:  0.942329
Epoch:  60 Loss: 0.27503 Valid accuracy:  0.942329
Epoch:  70 Loss: 0.31223 Valid accuracy:  0.942329
Epoch:  80 Loss: 0.358149 Valid accuracy:  0.940152
Epoch:  90 Loss: 0.415947 Valid accuracy:  0.939608


In [24]:
with tf.Session() as sess:
    saver.restore(sess, "./model/har_first_lstm_model.ckpt")
    valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
    test_acc = accuracy.eval(feed_dict={X: har_X_test, y: har_y_test})
    print("valid Accuracy: ", valid_acc)
    print("Test Accuracy: ", test_acc)

INFO:tensorflow:Restoring parameters from ./model/har_first_lstm_model.ckpt
valid Accuracy:  0.947769
Test Accuracy:  0.890397


## Basic RNN model with dropout

In [25]:
tf.reset_default_graph()
tf.set_random_seed(73)
np.random.seed(73)

X = tf.placeholder(tf.float32, (None, n_channels, seq_len), name="X")
y = tf.placeholder(tf.int32, (None), name="y")

# is in training phase
training = tf.placeholder_with_default(False, shape=(), name="training")


In [26]:
with tf.name_scope("rnn"):
    
    basic_cell = tf.contrib.rnn.BasicLSTMCell(num_units=n_neurons,
                                             activation=tf.nn.relu)
    drop_basic_cell = tf.contrib.rnn.DropoutWrapper(basic_cell, output_keep_prob=0.8)
    outputs, states = tf.nn.dynamic_rnn(drop_basic_cell, X, dtype=tf.float32)
    
    top_layer_h_state = states[-1]
    print(top_layer_h_state.shape)
    states_concat = tf.concat(axis=1, values=states)
    logits = tf.layers.dense(states_concat, n_classes)

(?, 100)


In [27]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy, name="loss")

In [28]:
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)

In [29]:
with tf.name_scope("eval"):
    #correct = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [30]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [31]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for i in range(n_batches):
            X_batch = X_train[i * batch_size:(i+1) * batch_size]
            y_batch = y_train[i * batch_size:(i+1) * batch_size]
            train_acc =  sess.run([training_op], feed_dict={X: X_batch, y: y_batch, training: True})
        
        if epoch % 10 == 0:
            valid_acc, loss_val = sess.run([accuracy, loss], feed_dict={X: X_valid, y: y_valid})
            print("Epoch: ", epoch,"Loss:", loss_val, "Valid accuracy: ", valid_acc)
    saver.save(sess, "./model/har_lstm_dropout_model.ckpt")

Epoch:  0 Loss: 1.15221 Valid accuracy:  0.634385
Epoch:  10 Loss: 0.193351 Valid accuracy:  0.926551
Epoch:  20 Loss: 0.193662 Valid accuracy:  0.934168
Epoch:  30 Loss: 0.20791 Valid accuracy:  0.934712
Epoch:  40 Loss: 0.213674 Valid accuracy:  0.936344
Epoch:  50 Loss: 0.223299 Valid accuracy:  0.93852
Epoch:  60 Loss: 0.239362 Valid accuracy:  0.940696
Epoch:  70 Loss: 0.259687 Valid accuracy:  0.940696
Epoch:  80 Loss: 0.290383 Valid accuracy:  0.942329
Epoch:  90 Loss: 0.324128 Valid accuracy:  0.937976


In [32]:
with tf.Session() as sess:
    saver.restore(sess, "./model/har_lstm_dropout_model.ckpt")
    valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
    test_acc = accuracy.eval(feed_dict={X: har_X_test, y: har_y_test})
    print("valid Accuracy: ", valid_acc)
    print("Test Accuracy: ", test_acc)

INFO:tensorflow:Restoring parameters from ./model/har_lstm_dropout_model.ckpt
valid Accuracy:  0.937976
Test Accuracy:  0.875467


# Three LSTM layers with  early stopping

In [33]:
# maximum number of epoch
n_epochs=600

In [34]:
tf.reset_default_graph()
tf.set_random_seed(73)
np.random.seed(73)

X = tf.placeholder(tf.float32, (None, n_channels, seq_len), name="X")
y = tf.placeholder(tf.int32, (None), name="y")

# is in training phase
training = tf.placeholder_with_default(False, shape=(), name="training")


In [35]:
with tf.name_scope("rnn"):
    
    cells = [tf.contrib.rnn.BasicLSTMCell(num_units=n_neurons, activation=tf.nn.relu)
                 for _ in range(n_layers)]
    
    multi_cells = tf.contrib.rnn.MultiRNNCell(cells)
   
    outputs, states = tf.nn.dynamic_rnn(multi_cells, X, dtype=tf.float32)
    top_layer_h_state = states[-1][-1]
    states_concat = tf.concat(axis=1, values=states)
    logits = tf.layers.dense(top_layer_h_state, n_classes)

In [36]:
top_layer_h_state.shape

TensorShape([Dimension(None), Dimension(100)])

In [37]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy, name="loss")

In [38]:
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)

In [39]:
with tf.name_scope("eval"):
    #correct = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [40]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [41]:
best_loss = np.infty
epochs_without_progress = 0
max_epochs_without_progress=150

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for i in range(n_batches):
            X_batch = X_train[i * batch_size:(i+1) * batch_size]
            y_batch = y_train[i * batch_size:(i+1) * batch_size]
            train_acc =  sess.run([training_op], feed_dict={X: X_batch, y: y_batch, training: True})
    
        if epoch % 5 == 0:
            valid_acc, loss_val = sess.run([accuracy, loss], feed_dict={X: X_valid, y: y_valid})
            print("Epoch: ", epoch, "Loss: ", loss_val, "Valid accuracy: ", valid_acc)
            if np.around(loss_val, decimals=4) < best_loss:
                saver.save(sess, "./model/har_lstm_early_stopping_model.ckpt")
                best_loss = loss_val
            else:
                epochs_without_progress +=5
                if epochs_without_progress > max_epochs_without_progress:
                    break

Epoch:  0 Loss:  1.42134 Valid accuracy:  0.433079
Epoch:  5 Loss:  0.262649 Valid accuracy:  0.895539
Epoch:  10 Loss:  0.252129 Valid accuracy:  0.920022
Epoch:  15 Loss:  0.218751 Valid accuracy:  0.930903
Epoch:  20 Loss:  0.243178 Valid accuracy:  0.930359
Epoch:  25 Loss:  0.215112 Valid accuracy:  0.937976
Epoch:  30 Loss:  0.215124 Valid accuracy:  0.94124
Epoch:  35 Loss:  0.216702 Valid accuracy:  0.940696
Epoch:  40 Loss:  0.228729 Valid accuracy:  0.939608
Epoch:  45 Loss:  0.247192 Valid accuracy:  0.939608
Epoch:  50 Loss:  0.259968 Valid accuracy:  0.940152
Epoch:  55 Loss:  0.299531 Valid accuracy:  0.939608
Epoch:  60 Loss:  0.194971 Valid accuracy:  0.941785
Epoch:  65 Loss:  0.188479 Valid accuracy:  0.943961
Epoch:  70 Loss:  0.198659 Valid accuracy:  0.942329
Epoch:  75 Loss:  0.209958 Valid accuracy:  0.945049
Epoch:  80 Loss:  0.219781 Valid accuracy:  0.940152
Epoch:  85 Loss:  0.232615 Valid accuracy:  0.943961
Epoch:  90 Loss:  0.243724 Valid accuracy:  0.9450

In [42]:
with tf.Session() as sess:
    saver.restore(sess, "./model/har_lstm_early_stopping_model.ckpt")
    valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
    test_acc = accuracy.eval(feed_dict={X: har_X_test, y: har_y_test})
    print("valid Accuracy: ", valid_acc)
    print("Test Accuracy: ", test_acc)

INFO:tensorflow:Restoring parameters from ./model/har_lstm_early_stopping_model.ckpt
valid Accuracy:  0.958107
Test Accuracy:  0.876824


## Bi-directional LSTM

In [43]:
n_epochs=200

In [44]:
tf.reset_default_graph()
tf.set_random_seed(73)
np.random.seed(73)

X = tf.placeholder(tf.float32, (None, n_channels, seq_len), name="X")
y = tf.placeholder(tf.int32, (None), name="y")

# is in training phase
training = tf.placeholder_with_default(False, shape=(), name="training")


In [45]:
with tf.name_scope("rnn"):
    
    fw_cells = [tf.contrib.rnn.BasicLSTMCell(num_units=n_neurons,
                                                 activation=tf.nn.relu)
                         for _ in range(n_layers)]
    
    bw_cells = [tf.contrib.rnn.BasicLSTMCell(num_units=n_neurons,
                                                 activation=tf.nn.relu)
                         for _ in range(n_layers)]
    multi_fw_cells = tf.contrib.rnn.MultiRNNCell(fw_cells)
    multi_bw_cells = tf.contrib.rnn.MultiRNNCell(bw_cells)
        
    outputs, states = tf.nn.bidirectional_dynamic_rnn(multi_fw_cells, multi_bw_cells, X, dtype=tf.float32)
    top_layer_h_state = states[-1][-1][-1]
    states_concat = tf.concat(axis=1, values=states)
    logits = tf.layers.dense(top_layer_h_state, n_classes)

In [46]:
top_layer_h_state.shape

TensorShape([Dimension(None), Dimension(100)])

In [47]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy, name="loss")

In [48]:
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)

In [49]:
with tf.name_scope("eval"):
    #correct = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [50]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [51]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for i in range(n_batches):
            X_batch = X_train[i * batch_size:(i+1) * batch_size]
            y_batch = y_train[i * batch_size:(i+1) * batch_size]
            train_acc =  sess.run([training_op], feed_dict={X: X_batch, y: y_batch, training: True})
             
        if epoch % 10 == 0:
            
            valid_acc, loss_val = sess.run([accuracy, loss], feed_dict={X: X_valid, y: y_valid})
            print("Epoch: ", epoch,"Loss:", loss_val, "Valid accuracy: ", valid_acc)
    saver.save(sess, "./model/har_lstm_model.ckpt")

Epoch:  0 Loss: 1.34852 Valid accuracy:  0.359086
Epoch:  10 Loss: 0.231776 Valid accuracy:  0.900435
Epoch:  20 Loss: 0.340472 Valid accuracy:  0.914037
Epoch:  30 Loss: 0.198543 Valid accuracy:  0.942329
Epoch:  40 Loss: 0.199508 Valid accuracy:  0.943417
Epoch:  50 Loss: 0.21916 Valid accuracy:  0.942329
Epoch:  60 Loss: 0.224513 Valid accuracy:  0.943961
Epoch:  70 Loss: 0.173894 Valid accuracy:  0.946681
Epoch:  80 Loss: 0.192409 Valid accuracy:  0.95049
Epoch:  90 Loss: 0.213671 Valid accuracy:  0.949946
Epoch:  100 Loss: 0.182285 Valid accuracy:  0.940696
Epoch:  110 Loss: 0.161961 Valid accuracy:  0.949946
Epoch:  120 Loss: 0.176571 Valid accuracy:  0.951034
Epoch:  130 Loss: 0.194259 Valid accuracy:  0.954298
Epoch:  140 Loss: 0.142518 Valid accuracy:  0.949946
Epoch:  150 Loss: 0.169103 Valid accuracy:  0.951034
Epoch:  160 Loss: 0.175119 Valid accuracy:  0.952666
Epoch:  170 Loss: 0.168235 Valid accuracy:  0.95049
Epoch:  180 Loss: 0.181946 Valid accuracy:  0.956474
Epoch:  

In [52]:
with tf.Session() as sess:
    saver.restore(sess, "./model/har_lstm_model.ckpt")
    valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
    test_acc = accuracy.eval(feed_dict={X: har_X_test, y: har_y_test})
    print("valid Accuracy: ", valid_acc)
    print("Test Accuracy: ", test_acc)

INFO:tensorflow:Restoring parameters from ./model/har_lstm_model.ckpt
valid Accuracy:  0.958107
Test Accuracy:  0.857482


# Bi-directional LSTM with dropout and early stopping

In [53]:
n_epochs=600

In [54]:
tf.reset_default_graph()
tf.set_random_seed(73)
np.random.seed(73)

X = tf.placeholder(tf.float32, (None, n_channels, seq_len), name="X")
y = tf.placeholder(tf.int32, (None), name="y")

# is in training phase
training = tf.placeholder_with_default(False, shape=(), name="training")


In [55]:
with tf.name_scope("rnn"):
    
    fw_cells = [tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.BasicLSTMCell(num_units=n_neurons, activation=tf.nn.relu),
                                             output_keep_prob=0.8)
                   for _ in range(n_layers)]
    bw_cells = [tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.BasicLSTMCell(num_units=n_neurons, activation=tf.nn.relu),
                                             output_keep_prob=0.8)
                   for _ in range(n_layers)]
    
    multi_fw_cells = tf.contrib.rnn.MultiRNNCell(fw_cells)
    multi_bw_cells = tf.contrib.rnn.MultiRNNCell(bw_cells)
    
    outputs, states = tf.nn.bidirectional_dynamic_rnn(multi_fw_cells, multi_bw_cells, X, dtype=tf.float32)
    top_layer_h_state = states[-1][-1][-1]
    states_concat = tf.concat(axis=1, values=states)
    logits = tf.layers.dense(top_layer_h_state, n_classes)

In [56]:
top_layer_h_state.shape

TensorShape([Dimension(None), Dimension(100)])

In [57]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy, name="loss")

In [58]:
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)

In [59]:
with tf.name_scope("eval"):
    #correct = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [60]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [61]:
best_loss = np.infty
epochs_without_progress = 0
max_epochs_without_progress=100

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for i in range(n_batches):
            X_batch = X_train[i * batch_size:(i+1) * batch_size]
            y_batch = y_train[i * batch_size:(i+1) * batch_size]
            train_acc =  sess.run([training_op], feed_dict={X: X_batch, y: y_batch, training: True})
    
        if epoch % 5 == 0:
            valid_acc, loss_val = sess.run([accuracy, loss], feed_dict={X: X_valid, y: y_valid})
            print("Epoch: ", epoch, "Loss: ", loss_val, "Valid accuracy: ", valid_acc)
            if np.around(loss_val, decimals=4) < best_loss:
                saver.save(sess, "./model/har_lstm_early_stopping_model.ckpt")
                best_loss = loss_val
            else:
                epochs_without_progress +=5
                if epochs_without_progress > max_epochs_without_progress:
                    break

Epoch:  0 Loss:  1.32307 Valid accuracy:  0.349837
Epoch:  5 Loss:  0.452492 Valid accuracy:  0.822089
Epoch:  10 Loss:  0.274729 Valid accuracy:  0.883025
Epoch:  15 Loss:  0.321143 Valid accuracy:  0.891186
Epoch:  20 Loss:  0.316844 Valid accuracy:  0.89173
Epoch:  25 Loss:  0.231183 Valid accuracy:  0.922742
Epoch:  30 Loss:  0.214636 Valid accuracy:  0.922742
Epoch:  35 Loss:  0.208267 Valid accuracy:  0.930903
Epoch:  40 Loss:  0.208984 Valid accuracy:  0.933079
Epoch:  45 Loss:  0.211385 Valid accuracy:  0.929815
Epoch:  50 Loss:  0.208526 Valid accuracy:  0.937976
Epoch:  55 Loss:  0.187079 Valid accuracy:  0.924918
Epoch:  60 Loss:  0.191786 Valid accuracy:  0.94124
Epoch:  65 Loss:  0.17694 Valid accuracy:  0.942873
Epoch:  70 Loss:  0.211531 Valid accuracy:  0.942329
Epoch:  75 Loss:  0.193638 Valid accuracy:  0.948313
Epoch:  80 Loss:  0.29349 Valid accuracy:  0.922198
Epoch:  85 Loss:  0.275174 Valid accuracy:  0.940152
Epoch:  90 Loss:  0.190795 Valid accuracy:  0.943417


In [62]:
with tf.Session() as sess:
    saver.restore(sess, "./model/har_lstm_early_stopping_model.ckpt")
    valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
    test_acc = accuracy.eval(feed_dict={X: har_X_test, y: har_y_test})
    print("valid Accuracy: ", valid_acc)
    print("Test Accuracy: ", test_acc)

INFO:tensorflow:Restoring parameters from ./model/har_lstm_early_stopping_model.ckpt
valid Accuracy:  0.949401
Test Accuracy:  0.891754
